In [1]:
import jieba
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS



In [ ]:
gender_labels = {0: "Unknown", 1: "Male", 2: "Female"}
sentiment_label = {0:"Disgusted",0.2:"Dissatisfied",0.4:"Indifferent",0.6:"Satisfied",0.8:"Delighted"}
# 读取数据
csvpath = "D:\\information_information_visualization_HW\\final_project\\public\\data\\Nezha2_neo_building_clouds.csv"
df = pd.read_csv(csvpath, encoding='utf-8-sig', low_memory=False)



In [4]:
print(df.head())

   comment_id  score   content             time city  gender  sentiment  \
0  1211665559    5.0   二刷！依旧精彩  2025/4/21 21:16   佛山       0   0.920307   
1  1211652698    5.0  支持一下国产动漫  2025/4/21 21:15   延边       1   0.513423   
2  1211623835    5.0   剧情搞笑又紧张  2025/4/21 21:13   武汉       0   0.318905   
3  1211642772    3.5  还行吧，特效不错  2025/4/21 21:08   深圳       0   0.207190   
4  1211659669    4.5    值得看的一部  2025/4/21 20:59   海东       0   0.835402   

  province  gender_mapped  
0      广东省              0  
1     未知省份              1  
2      湖北省              0  
3      广东省              0  
4      青海省              0  


In [5]:
custom_words = {
    "trigger_full": [
        # 内容/设定/文化元素
        "国产动画", "国产", "中国", "传统文化", "神话", "改编", "家庭关系", "亲情", "友情", "成长", "命运", "传承", "英雄", "勇气",
        # 角色与地名
        "哪吒", "敖丙", "龙王", "李靖", "申公豹", "太乙真人", "东海龙王", "西海龙王", "南海龙王", "北海龙王", "陈塘关",
        # 制作与风格
        "国漫", "3D动画", "视觉效果", "配音", "特效制作", "打斗场面", "动作设计", "原画", "配乐",
        # 观影相关
        "电影院", "春节", "亲子", "孩子", "孩子喜欢", "家人", "一起", "家庭观影", "观影", "体验",
        # 反馈标签
        "适合家庭", "适合孩子", "老少皆宜", "寓教于乐", "感情充沛"
    ],

    "agenda_full": [
        # 性别议题
        "女性形象", "女性角色", "女权", "性别平等", "性别刻板印象", "女性独立", "刻画女性", "歧视女性", "尊重女性", "女性代表性",
        # 国家/文化身份
        "中国动画", "国产动画", "国漫崛起", "文化输出", "文化自信", "中国风", "中国故事", "民族自豪感", "本土文化", "传统文化",
        # 教育/价值观
        "家庭教育", "亲子教育", "价值观输出", "儿童成长", "教育意义", "正向引导", "榜样作用", "儿童启蒙", "情感教育",
        # 社会现实议题
        "内卷", "焦虑", "打工人", "社会压抑", "情绪共鸣", "代际关系", "阶级差距", "环境污染", "城乡差距", "中年危机",
        # 制作与审美
        "抄袭", "致敬", "原创", "工业化制作", "资本运作", "视觉内卷", "工业水准", "审美疲劳", "制作粗糙", "动画工业",
        # 舆论机制
        "豆瓣评分", "水军", "口碑翻车", "评分虚高", "营销过度", "公关", "热搜", "带节奏", "被洗脑", "情绪操控",
        # 中外对比
        "迪士尼", "皮克斯", "梦工厂", "好莱坞动画", "国外动画", "外国动画", "西方叙事", "迪士尼叙事", "文化差异",
        "中美差距", "制作差距", "工业水准差距", "国际对比", "对标迪士尼", "媲美皮克斯", "赶超国外动画", "超过玩具总动员", "不输冰雪奇缘"
    ],

    "agenda_split": [
        "女性", "女权", "性别", "刻板", "形象", "角色", "尊重", "歧视", "独立", "代表性",
        "中国", "国产", "国漫", "崛起", "文化", "自信", "输出", "中国风", "故事", "本土", "民族", "传统",
        "家庭", "亲子", "教育", "价值观", "成长", "儿童", "榜样", "启蒙", "引导", "情感",
        "内卷", "焦虑", "打工人", "压抑", "共鸣", "代际", "阶级", "差距", "环境", "污染", "中年", "危机",
        "抄袭", "致敬", "原创", "工业化", "资本", "运作", "视觉", "水准", "疲劳", "粗糙", "工业",
        "豆瓣", "评分", "水军", "口碑", "翻车", "虚高", "营销", "过度", "公关", "热搜", "节奏", "洗脑", "操控", "情绪",
        "迪士尼", "皮克斯", "梦工厂", "好莱坞", "国外", "外国", "西方", "对标", "媲美", "赶超", "不输", "超过", "差距",
        "玩具总动员", "冰雪奇缘", "熊出没", "叙事", "国际"
    ],

    "entities": [
        # 人物
        "哪吒", "敖丙", "李靖", "太乙真人", "申公豹", "龙王", "哪吒三太子", "石矶娘娘", "哪吒闹海",
        "东海龙王", "夜叉", "雷震子", "金吒", "木吒", "托塔天王", "哪吒之魔童降世", "哪吒重生",
        # 地点
        "东海", "陈塘关", "九重天", "天庭", "地府", "龙宫", "人间", "三界", "凡间", "蓬莱", "灵山", "南天门"
    ]
}



# 批量添加所有需要召回的词
for group in custom_words.values():
    for word in group:
        jieba.add_word(word)


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\顾子琪\AppData\Local\Temp\jieba.cache
Loading model cost 0.622 seconds.
Prefix dict has been built successfully.


In [ ]:
# 分词函数
def segment_words(text, stopwords):
    words = jieba.cut(text, cut_all=True)
    return " ".join([word for word in words if word not in stopwords and len(word.strip()) > 1])

sentiment_label = {
    0: "Disgusted",
    1: "Dissatisfied",
    2: "Indifferent",
    3: "Satisfied",
    4: "Delighted"
}

for gender in [0, 1, 2]:
    j = 0
    for idx, i in enumerate(np.arange(0, 1.0, 0.2)):
        low, high = round(i, 2), round(i + 0.2, 2)

        selected_comments = df[
            (df['sentiment'] >= low) & (df['sentiment'] < high) & (df['gender'] == gender)
        ]['content'].dropna().astype(str)

        if selected_comments.empty:
            print(f"[跳过] gender {gender}，情绪区间 {low} ~ {high} 内无评论")
            continue

        print(f"[处理] gender {gender}，情绪区间：{low} ~ {high}，评论数：{len(selected_comments)}")

        words = " ".join(selected_comments.apply(lambda x: segment_words(x, stopwords)))

        wc = WordCloud(
            width=800,
            height=600,
            background_color="white",
            font_path='C:/Windows/Fonts/simhei.ttf',
            max_words=50
        ).generate(words)

        plt.figure(figsize=(10, 7.5))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')
        plt.title(f"{gender_labels[gender]} feeling {sentiment_label[idx]}", fontsize=18, pad=20)
        # plt.show()
        output_path = f"D:/information_information_visualization_HW/final_project/public/data/Gender{gender}Sentiment{j}.png"
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ 带标题的词云已保存到：{output_path}")
        j += 1

In [ ]:
# 添加自定义词
trigger_keywords = [
    # 内容/设定/文化元素
    "国产动画","国产", "中国", "传统文化", "神话", "神话", "改编", "家庭关系", "亲情", "友情", "成长", "命运", "传承", "英雄", "勇气",
    # 角色和地名元素
    "哪吒", "敖丙", "龙王", "李靖", "申公豹", "太乙真人", "龙王", "东海龙王", "西海龙王", "南海龙王", "北海龙王", "陈塘关"
    # 制作与风格
    "国漫", "3D动画", "视觉效果", "配音", "特效制作", "打斗场面", "动作设计", "原画", "配乐",
    # 观影相关
    "电影院", "春节", "亲子", "孩子", "孩子喜欢", "家人", "一起", "家庭观影", "观影", "体验"
    # 观众反馈标签
    "适合家庭", "适合孩子", "老少皆宜", "寓教于乐", "感情充沛"
]

agenda_words_full = [
    # 性别议题
    "女性形象", "女性角色", "女权", "性别平等", "性别刻板印象", "女性独立", "刻画女性", "歧视女性", "尊重女性", "女性代表性",
    
    # 国家/文化身份
    "中国动画", "国产动画", "国漫崛起", "文化输出", "文化自信", "中国风", "中国故事", "民族自豪感", "本土文化", "传统文化",
    
    # 教育/价值观
    "家庭教育", "亲子教育", "价值观输出", "儿童成长", "教育意义", "正向引导", "榜样作用", "儿童启蒙", "情感教育",

    # 社会现实议题
    "内卷", "焦虑", "打工人", "社会压抑", "情绪共鸣", "代际关系", "阶级差距", "环境污染", "城乡差距", "中年危机",

    # 产业/审美/制作层面
    "抄袭", "致敬", "原创", "工业化制作", "资本运作", "视觉内卷", "工业水准", "审美疲劳", "制作粗糙", "动画工业",

    # 舆论生态/评论机制
    "豆瓣评分", "水军", "口碑翻车", "评分虚高", "营销过度", "公关", "热搜", "带节奏", "被洗脑", "情绪操控"
]

agenda_words_split = [
    # 性别
    "女性", "女权", "性别", "刻板", "形象", "角色", "尊重", "歧视", "独立", "代表性",
    # 国家文化
    "中国", "国产", "国漫", "崛起", "文化", "自信", "输出", "中国风", "故事", "本土", "民族", "传统",
    # 教育
    "家庭", "亲子", "教育", "价值观", "成长", "儿童", "榜样", "启蒙", "引导", "情感",
    # 社会议题
    "内卷", "焦虑", "打工人", "压抑", "共鸣", "代际", "阶级", "差距", "环境", "污染", "中年", "危机",
    # 制作审美
    "抄袭", "致敬", "原创", "工业化", "资本", "运作", "视觉", "水准", "疲劳", "粗糙", "工业",
    # 舆论评论
    "豆瓣", "评分", "水军", "口碑", "翻车", "虚高", "营销", "过度", "公关", "热搜", "节奏", "洗脑", "操控", "情绪"
]

agenda_words_full += [
    # 中外动画对比
    "迪士尼", "皮克斯", "梦工厂", "好莱坞动画", "国外动画", "外国动画", "西方叙事", "迪士尼叙事", "文化差异",
    "中美差距", "制作差距", "工业水准差距", "国际对比", "对标迪士尼", "媲美皮克斯", "赶超国外动画", "超过玩具总动员", "不输冰雪奇缘"
]

agenda_words_split += [
    "迪士尼", "皮克斯", "梦工厂", "好莱坞", "国外", "外国", "西方", "对标", "媲美", "赶超", "不输", "超过", "差距",
    "玩具总动员", "冰雪奇缘", "熊出没", "文化", "叙事", "工业", "水准", "国际"
]


character_words = [
    "哪吒", "敖丙", "李靖", "太乙真人", "申公豹", "龙王", "哪吒三太子", "石矶娘娘", "哪吒闹海", 
    "东海龙王", "夜叉", "雷震子", "金吒", "木吒", "托塔天王", "哪吒之魔童降世", "哪吒重生"
]

location_words = [
    "东海", "陈塘关", "九重天", "天庭", "地府", "龙宫", "人间", "三界", "凡间", "蓬莱", "灵山", "南天门"
]



for word in custom_words:
    jieba.add_word(word)



#添加自定义停用词：
custom_stopwords = [
    # 情感副词/程度副词
    "非常", "特别", "超级", "挺", "有点", "真的", "简直", "太棒", "太棒了", "太好了", "还行", "不错", "挺好", "很快", "可以", "不行","有点", "还可以", "真不错"

    # 模糊指代/没有主题意义
    "我们", "你们", "他们", "自己", "大家", "这种", "这个", "那里", "啥", "多少", "一部", "第一部", "第二部", "第三部",
    
    #表达动作的单词
    "支持", "推荐", "值得", "必须"
]

# 构建停用词表
stopwords = STOPWORDS.copy()
stopwords.update({
    "电影", "最后", "就是", "不过", "这个", "一个", "感觉", "这部", 
    "虽然", "不是", "真的", "觉得", "还是", "但是", "好看", "总动员", "玩具", "奇缘", "冰雪","非常","喜欢"
})

# 分词函数
def segment_words(text, stopwords):
    words = jieba.cut(text, cut_all=True)
    return " ".join([word for word in words if word not in stopwords and len(word.strip()) > 1])

sentiment_label = {
    0: "Disgusted",
    1: "Dissatisfied",
    2: "Indifferent",
    3: "Satisfied",
    4: "Delighted"
}

for gender in [0, 1, 2]:
    j = 0
    for idx, i in enumerate(np.arange(0, 1.0, 0.2)):
        low, high = round(i, 2), round(i + 0.2, 2)

        selected_comments = df[
            (df['sentiment'] >= low) & (df['sentiment'] < high) & (df['gender'] == gender)
        ]['content'].dropna().astype(str)

        if selected_comments.empty:
            print(f"[跳过] gender {gender}，情绪区间 {low} ~ {high} 内无评论")
            continue

        print(f"[处理] gender {gender}，情绪区间：{low} ~ {high}，评论数：{len(selected_comments)}")

        words = " ".join(selected_comments.apply(lambda x: segment_words(x, stopwords)))

        wc = WordCloud(
            width=800,
            height=600,
            background_color="white",
            font_path='C:/Windows/Fonts/simhei.ttf',
            max_words=50
        ).generate(words)

        plt.figure(figsize=(10, 7.5))
        plt.imshow(wc, interpolation='bilinear')
        plt.axis('off')
        plt.title(f"{gender_labels[gender]} feeling {sentiment_label[idx]}", fontsize=18, pad=20)
        # plt.show()
        output_path = f"D:/information_information_visualization_HW/final_project/public/data/neo_wordcloud/Gender{gender}Sentiment{j}.png"
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ 带标题的词云已保存到：{output_path}")
        j += 1